In [60]:
import pandas as pd
import numpy as np

In [61]:
import pyprind
import pandas as pd
import os
import sys


basepath = "/home/hscuser/nlp/aclImdb_v1/aclImdb"
labels = {'pos':1, 'neg':0}
pbar = pyprind.ProgBar(50000, stream = sys.stdout)
df = pd.DataFrame()
df_reviews = []
df_sentiments = []
for s in ("test", "train"):
    for l in ("pos", "neg"):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 'r', encoding = 'utf-8') as infile:
                txt = infile.read()
            df_reviews.append(txt)
            df_sentiments.append(labels[l])
            pbar.update()
df['review'] = df_reviews
df['sentiment'] = df_sentiments
# df.columns = ['review','sentiment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


In [62]:
len(df_reviews), len(df_sentiments)

(50000, 50000)

In [63]:
df.head(4)

,review,sentiment
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1
3,"I saw this film in a sneak preview, and it is ...",1


In [64]:
df.shape

(50000, 2)

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


In [66]:
df.describe()

,sentiment
count,50000.000000
mean,0.500000
std,0.500005
min,0.000000
25%,0.000000
50%,0.500000
75%,1.000000
max,1.000000


In [67]:
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('movie_data.csv', index=False, encoding='utf-8')

### Introducing bag-of-words

In [68]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(ngram_range=(1,1))
docs = np.array(['The sun is shining is', 'The weather is sweet', 'The sun in shining, the weather is sweet', 'and one and one is two'])
bag = count.fit_transform(docs)

In [69]:
vocab = count.vocabulary_
print(vocab)
inv_vocab = {}
for key, value in vocab.items():
    inv_vocab[value] = key
print(inv_vocab)

{'the': 7, 'sun': 5, 'is': 2, 'shining': 4, 'weather': 9, 'sweet': 6, 'in': 1, 'and': 0, 'one': 3, 'two': 8}
{7: 'the', 5: 'sun', 2: 'is', 4: 'shining', 9: 'weather', 6: 'sweet', 1: 'in', 0: 'and', 3: 'one', 8: 'two'}


In [70]:
dump = bag.toarray()

In [71]:
dump

array([[0, 0, 2, 0, 1, 1, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 1, 1, 0, 1],
       [0, 1, 1, 0, 1, 1, 1, 2, 0, 1],
       [2, 0, 1, 2, 0, 0, 0, 0, 1, 0]])

In [72]:
dump.shape

(4, 10)

In [73]:
for i in range(dump.shape[0]):
    for j in range(dump.shape[1]):
        if dump[i][j] >= 1:
            print(f" {inv_vocab[j]} : {dump[i][j]}", end=",")
    print()

 is : 2, shining : 1, sun : 1, the : 1,
 is : 1, sweet : 1, the : 1, weather : 1,
 in : 1, is : 1, shining : 1, sun : 1, sweet : 1, the : 2, weather : 1,
 and : 2, is : 1, one : 2, two : 1,


## Term frequency - inverse document frequency 

In [74]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(
    use_idf = True,
    norm = 'l2',
    smooth_idf = True
)

np.set_printoptions(precision = 2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.   0.63 0.   0.48 0.48 0.   0.39 0.   0.  ]
 [0.   0.   0.38 0.   0.   0.   0.57 0.46 0.   0.57]
 [0.   0.43 0.22 0.   0.34 0.34 0.34 0.55 0.   0.34]
 [0.66 0.   0.17 0.66 0.   0.   0.   0.   0.33 0.  ]]


## cleaning text data

In [75]:
df.loc[1, 'review']#[-50:]

'Actor turned director Bill Paxton follows up his promising debut, the Gothic-horror "Frailty", with this family friendly sports drama about the 1913 U.S. Open where a young American caddy rises from his humble background to play against his Bristish idol in what was dubbed as "The Greatest Game Ever Played." I\'m no fan of golf, and these scrappy underdog sports flicks are a dime a dozen (most recently done to grand effect with "Miracle" and "Cinderella Man"), but some how this film was enthralling all the same.<br /><br />The film starts with some creative opening credits (imagine a Disneyfied version of the animated opening credits of HBO\'s "Carnivale" and "Rome"), but lumbers along slowly for its first by-the-numbers hour. Once the action moves to the U.S. Open things pick up very well. Paxton does a nice job and shows a knack for effective directorial flourishes (I loved the rain-soaked montage of the action on day two of the open) that propel the plot further or add some unexpec

In [76]:
# if we see above cell there were many html tags which we does not need. we need to remove them.
# sometimes there might be some emotions like smilie's like :), they add/ help us in the sentiment analysis so we need to keep them.


In [77]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', 
                            text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-',''))
    return text

In [78]:
len(preprocessor(df['review'][1]))

1942

In [79]:
len(df['review'][1])

2004

In [80]:
df['review'] = df['review'].apply(preprocessor)

## processing documents into tokens

In [81]:
## afer cleaning the data, we need to represent the data in the form that our model understand that is called as tokenization
## to do tokenization, for now we use the .split() method for the string which separates the words with whitespace

In [82]:
def tokenizer(text):
    return text.split()

In [83]:
tokenizer("hey how are you today")

['hey', 'how', 'are', 'you', 'today']

In [84]:
## there is also a usefull techinique is word stemming, which is the process of transforming a word into its root form.
## it allows us to map related words to the same stem.
## this stemming algo is called as the porter stemming algo
## this is available in the nltk lib

In [85]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()


In [86]:
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]
tokenizer_porter("running is good for health")

['run', 'is', 'good', 'for', 'health']

In [87]:
import nltk

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/hscuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [88]:
from nltk.corpus import stopwords

stop = stopwords.words("english")
[ w for w in tokenizer_porter('a runner likes runnning and runs a lot') if w not in stop]

['runner', 'like', 'runn', 'run', 'lot']

In [89]:
## Training a logistic regression model for document classification

In [90]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [91]:
# use a gridsearchcv to find a optimal set of parameters for our logistic regression model using 5-fold stratified cross-validation

In [92]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents = None,
                        lowercase = False,
                        preprocessor = None)
small_param_grid = [
    {
        'vect__ngram_range': [(1,1)],
        'vect__stop_words': [None],
        'vect__tokenizer':[tokenizer, tokenizer_porter],
        'clf__penalty':['l2'],
        'clf__C':[1.0, 10.0]
    },
    {
        'vect__ngram_range': [(1,1)],
        'vect__stop_words': [None],
        'vect__tokenizer':[tokenizer, tokenizer_porter],
        'vect__use_idf': [False],
        'vect__norm':[None],
        'clf__penalty':['l2'],
        'clf__C':[1.0, 10.0]
    },
]
lr_tfidf = Pipeline([
    ('vect', tfidf),
    ('clf', LogisticRegression(solver='liblinear'))
     ])
gs_lr_tfidf = GridSearchCV(lr_tfidf, small_param_grid, scoring = 'accuracy', cv=5, verbose=2, n_jobs=1)
gs_lr_tfidf.fit(X_train, y_train)

/home/hscuser/nlp/env/nllp/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x78a20521ce50>; total time=   1.7s


/home/hscuser/nlp/env/nllp/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x78a20521ce50>; total time=   1.6s


/home/hscuser/nlp/env/nllp/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


...

[CV] END clf__C=10.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__stop_words=None, vect__tokenizer=<function tokenizer_porter at 0x78a205644040>, vect__use_idf=False; total time=  45.5s


/home/hscuser/nlp/env/nllp/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__C=10.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__stop_words=None, vect__tokenizer=<function tokenizer_porter at 0x78a205644040>, vect__use_idf=False; total time=  45.3s


/home/hscuser/nlp/env/nllp/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,estimator,Pipeline(step...liblinear'))])
,param_grid,"[{'clf__C': [1.0, 10.0], 'clf__penalty': ['l2'], 'vect__ngram_range': [(1, ...)], 'vect__stop_words': [None], ...}, {'clf__C': [1.0, 10.0], 'clf__penalty': ['l2'], 'vect__ngram_range': [(1, ...)], 'vect__norm': [None], ...}]"
,scoring,'accuracy'
,n_jobs,1
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,input,'content'


In [93]:
print(f'Best parameter set: {gs_lr_tfidf.best_params_}')

Best parameter set: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x78a20521ce50>}


In [94]:
!pip install jupyter_contrib_nbextensions

In [95]:
print(f'CV Accuracy: {gs_lr_tfidf.best_score_:.3f}')
clf = gs_lr_tfidf.best_estimator_
print(f'Test accuracy: {clf.score(X_test, y_test):.3f}')

CV Accuracy: 0.891
Test accuracy: 0.897


In [96]:
### need to learn naive bayes classifier

## working with bigger data - online algorithms and out-of-core learning

In [97]:
import numpy as np
import re
from nltk.corpus import stopwords

stop = stopwords.words(
    'english'
)

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

In [98]:
def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [99]:
next(stream_docs(path='movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [100]:
def get_minibatch(doc_stream, size):
    docs, y =[], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs,y

In [101]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
vect = HashingVectorizer(decode_error = 'ignore',
                        n_features = 2**21,
                         tokenizer = tokenizer,
                         preprocessor = None
                        )
clf = SGDClassifier(loss='log_loss', random_state=1)
doc_stream = stream_docs(path = "movie_data.csv")

In [102]:
import pyprind
pbar = pyprind.ProgBar(45)
classes = np.array([0,1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size = 1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:18


In [103]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print(f'Accuracy: {clf.score(X_test, y_test):.3f}')


Accuracy: 0.868


In [104]:
### need to understand the word2vec model

In [105]:
clf = clf.partial_fit(X_test, y_test)

In [106]:
## topic modeling with latent dirichlet allocation

In [107]:
import pandas as pd

df = pd.read_csv('movie_data.csv',encoding='utf-8')
# df = df.rename(columns={})

In [108]:
df.head()

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


In [110]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words = 'english', max_df=1, max_features=5000)

X = count.fit_transform(df['review'].values)

In [115]:
len(count.vocabulary_)

5000

In [116]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components = 10, random_state= 123, learning_method='batch')

X_topics = lda.fit_transform(X)

In [117]:
lda.components_.shape

(10, 5000)

In [120]:
n_top_words = 5

feature_names = count.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    print(f'Topic {(topic_idx + 1)}:')
    print(' '.join([feature_names[i] for i in topic.argsort()[:-n_top_words-1:-1]]))
    

Topic 1:
trivialboring kazaf liliom weedon jeffs
Topic 2:
prag blacksnake cabell anouska lesseos
Topic 3:
keitaro tronje moores hollins sampredo
Topic 4:
shlomi floraine hayakawa hongshen norliss
Topic 5:
fanshawe lematt shinae gant kazuhiro
Topic 6:
uzumakis tucsos narvo tmtm chassidic
Topic 7:
hanka rosarios perú moag crystina
Topic 8:
tetsurô bazza iameracing tagge jouissance
Topic 9:
montero tobell numar ramar tsa
Topic 10:
allana havers rf frewer chise


In [121]:
horror = X_topics[:,5].argsort()[::-1]

for iter_idx, movie_idx in enumerate(horror[:3]):
    print(f"\n Horror Movie #{(iter_idx+1)}")
    print(df['review'][movie_idx][:300], "...")


 Horror Movie #1
For me the only reason for having a look at this remake was to see how bad and funny it could be. There was no doubt about it being funny and bad, because I had seen "Voyna i mir" (1968). Shall we begin? Here we go...<br /><br />Robert Dornhelm & Brendan Donnison's Pierre Bezukhov - a lean fellow th ...

 Horror Movie #2
"Only the Valiant" qualifies as a gritty good western. This Gregory Peck cavalry versus the Indians oater is a solemn suicide mission without a trace of humor. Veteran director Gordon Douglas has helmed a grim, harrowing outdoors epic with an ideal cast of tough guys under considerable pressure; eve ...

 Horror Movie #3
Uzumaki, which translates into "spirals", arriving within this new wave of Asian Horror films following such hits like Ringu, Ju-On and The Eye (two of them with remakes.. and much more coming like Dark Water and Tale of Two Sisters), falls short of the spooky, supernatural thriller element so chara ...
